## https://realpython.com/modern-web-automation-with-python-and-selenium/

In [28]:
import datetime
import logging
import numpy as np
import os
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
# # set path to current working directory for cron job
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

# logger = logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
# formatter = logging.Formatter('%(asctime)s:%(name)s:%(message)s')
# file_handler = logging.FileHandler('scraper.log')
# file_handler.setFormatter(formatter)
# logger.addHandler(file_handler)

In [3]:
# import logging
# from logging.handlers import RotatingFileHandler

# logger = logging.getLogger('my_logger')
# handler = RotatingFileHandler('my_log.log', maxBytes=2000, backupCount=10)
# logger.addHandler(handler)

In [5]:
CITIES = None
SAVE_DIRECTORY = 'daily_scrape'

In [47]:
class CityScraper:
    """Scrapes a city for apartment listings."""
    
    def __init__(self, city, save_directory='scraped'):
        self.city = city
        self.city_url = f'https://www.trulia.com/for_rent/{city}'
        self.browser = self._set_browser()

    def _set_browser(self):
        firefox_options = webdriver.FirefoxOptions()
        firefox_options.set_headless()
        browser = webdriver.Firefox(firefox_options=firefox_options)
        return browser

    def browser_get(self, url):
        """ Gets the url specified.  Contains error handling"""
        try:
            self.browser.get(url)
        except (ConnectionError, ConnectionResetError) as e:
            logger.info(f'Error {e} for URL: {url}')

    def get_list_page_urls(self):
        """Gets the urls on the list page"""
        listing_urls = []
        for elem in self.browser.find_elements_by_class_name('jLNYlr'):
            try:
                listing_urls.append(elem.find_element_by_tag_name('a').get_attribute('href'))
            except NoSuchElementException:
                continue
        return listing_urls

    def get_next_page(self):
        """ Gets next page link and returns flag indicating if last page"""
        try:
            # next_page = self.browser.find_element_by_class_name('bpESQu').get_attribute('href')
            soup = BeautifulSoup(self.browser.page_source, 'html.parser')
            href_suffix = soup.find('a', {'aria-label': 'Next Page'})['href']
            next_page = 'https://www.trulia.com' + href_suffix
            last_page = False
            print(f'HREF SUFFIX: {href_suffix}')
            return next_page, last_page
        except TypeError:
            next_page = None
            last_page = True
            return next_page, last_page

    def get_apartment_urls_for_city(self):
        '''Gets a list of urls for city from all listing pages'''
        i = 1
        url_list = []
        next_page = self.city_url
        last_page = False
        while last_page is False:
        # for _ in range(1):
            print(f'Page {i}, Total Apartment URLs: {len(url_list)}') if i % 10 == 0 else None
            self.browser_get(next_page)

            list_page_urls = self.get_list_page_urls()
            url_list.extend(list_page_urls)
            print(f'Current URL: {self.browser.current_url}')
            print(f'Last page: {last_page}')
            next_page, last_page = self.get_next_page()
            print(f'NEW Next page: {next_page}')
            print(f'NEW Last page: {last_page}')
            print(f'Iteration: {i}')
            i += 1
            time.sleep(.05)
        return url_list

    def get_apartment_data(self, url):
        '''Gets apartment data for the url specified'''
        self.browser_get(url)
        content = self.browser.page_source
        soup = BeautifulSoup(content, 'html.parser')

        apartment_list = []

        for floor_plan_table in soup.find_all('table', {'data-testid': 'floor-plan-group'}):
            for tr in floor_plan_table.find_all('tr'):

                unit = tr.find('div', {'color': 'highlight'}).text

                sqft = tr.find('td', {'class': 'FloorPlanTable__FloorPlanFloorSpaceCell-sc-1ghu3y7-5'}).text

                bed = tr.find_all('td', {'class': 'FloorPlanTable__FloorPlanFeaturesCell-sc-1ghu3y7-4'})[0].text

                bath = tr.find_all('td', {'class': 'FloorPlanTable__FloorPlanFeaturesCell-sc-1ghu3y7-4'})[1].text

                price = tr.find_all('td', {'class': 'FloorPlanTable__FloorPlanCell-sc-1ghu3y7-2',
                                        'class': 'FloorPlanTable__FloorPlanSMCell-sc-1ghu3y7-8'},
                                    limit=2)[1].text

                name = soup.find('span', {'data-testid': 'home-details-summary-headline'}).text

                address = soup.find_all('span', {'data-testid': 'home-details-summary-city-state'})[0].text

                city_state_zip = soup.find_all('span', {'data-testid': 'home-details-summary-city-state'})[1].text

                city, state, zipcode = city_state_zip.replace(',', '').rsplit(maxsplit=2)

                description = soup.find('div', {'data-testid': 'home-description-text-description-text'}).text

                details = [detail.text for detail in soup.find_all(
                    'li', {'class': 'FeatureList__FeatureListItem-iipbki-0'}
                )]
                details = ' ,'.join(details)

                apartment_url = url
                date = str(datetime.datetime.now().date())
                apartment_list.append([name, address, unit, sqft, bed, bath, price, city,
                                    state, zipcode, description, details, apartment_url, date])
        return apartment_list

    def create_apartment_df(self, data):
        df = pd.DataFrame(data, columns=['name', 'address', 'unit', 'sqft', 'bed', 'bath', 'price',
                               'city', 'state', 'zipcode', 'description', 'details', 'url', 'date'])
        return df

    def clean_apartment_df(self, df):
        '''Formats the dataframe to remove special characters, spaces, and NaN values.
        Removes units with price ranges, rather than one specified price.'''

        df.sqft = df.sqft.str.replace('sqft', '').str.replace(',', '').str.strip()
        df = df.loc[df.sqft != ''] # throw out empty square foot apts
        mask = df.sqft.str.contains('-')
        df.loc[mask, 'sqft'] = df.loc[mask, 'sqft'].apply(lambda x: np.mean(list(map(int, x.split('-')))))
        df.price = df.price.str.replace('Contact', '')
        df.price = df.price.str.replace('$', '').str.replace(',', '').str.replace('+', '').str.strip()
        df.bath = df.bath.str.replace('ba', '').str.strip()
        df.bed = df.bed.str.replace('bd', '').str.lower().replace('studio', 0).str.strip()
        df.bed = df.bed.replace(np.nan, 0)
        df = df[~df.price.str.contains('-', na=False)]  # throw out price range apts
        df.replace(' ', '', inplace=True)  # whitespace to blank
        df.replace('', np.nan, inplace=True)  # blank to NaN
        # df.dropna(inplace=True)  # drop NaN rows
        
        return df

    def convert_df_columns(self, df):
        '''Converts rows to numeric and float for calculations'''
        df = df.astype({'sqft': 'int32', 'price': 'int32', 'bath': 'float32', 
                        'bed': 'float32', 'zipcode': 'int32'})
        return df

    def save_to_csv(self, df):
        '''Saves cleaned dataframe to csv file in "daily_scrape_files" folder"'''
        scraped_date = str(datetime.datetime.now().date())
        city_name = self.city.replace(',', '_')
        filedir = f'{SAVE_DIRECTORY}/{city_name}'
        os.makedirs(filedir, exist_ok=True)
        df.to_csv(f'{filedir}/{scraped_date}.csv', index=False)

In [48]:
cities = ['Woburn,MA']

for city in cities:
    print(f'START scraping {city}')    
    scraper = CityScraper(city, save_directory=SAVE_DIRECTORY)
    
    print('Getting URL list')
    apartment_url_list = scraper.get_apartment_urls_for_city()
    print(f'URLs retrieved: {len(apartment_url_list)}')
    
    print('Getting apartment data from url_list')
    apartments_data = []
    for i, apartment_url in enumerate(tqdm(apartment_url_list), start=1):
        print(f'URL {i} of {len(apartment_url_list)}') if i % 500 == 0 else None
        try:
            apt_data = scraper.get_apartment_data(apartment_url)
            apartments_data.extend(apt_data)
        except Exception as e:
            print(f'Exception getting apartment data for url {apartment_url}', e)
            # logger.exception('Error adding data to list', e)
            continue
    print(f'Apartments retrieved: {len(apartments_data)}')
    
    apartment_df = scraper.create_apartment_df(apartments_data)
    cleaned_apartment_df = scraper.clean_apartment_df(apartment_df)
    cleaned_and_converted_apartment_df = scraper.convert_df_columns(cleaned_apartment_df)
    scraper.save_to_csv(cleaned_and_converted_apartment_df)
    print(f'FINISH scraping {city}')
    print()

START scraping Woburn,MA
Getting URL list
Current URL: https://www.trulia.com/for_rent/Woburn,MA/
Last page: False
HREF SUFFIX: /for_rent/Woburn,MA/2_p/
NEW Next page: https://www.trulia.com/for_rent/Woburn,MA/2_p/
NEW Last page: False
Iteration: 1
Current URL: https://www.trulia.com/for_rent/Woburn,MA/2_p/
Last page: False
HREF SUFFIX: /for_rent/Woburn,MA/3_p/
NEW Next page: https://www.trulia.com/for_rent/Woburn,MA/3_p/
NEW Last page: False
Iteration: 2
Current URL: https://www.trulia.com/for_rent/Woburn,MA/3_p/
Last page: False
NEW Next page: None
NEW Last page: True
Iteration: 3
  0%|          | 0/92 [00:00<?, ?it/s]URLs retrieved: 92
Getting apartment data from url_list
100%|██████████| 92/92 [01:37<00:00,  1.06s/it]Apartments retrieved: 175
FINISH scraping Woburn,MA




In [ ]:
aria-label="Next Page"

In [18]:
scc = CityScraper('Cambridge,MA')


# cleaned = scc.clean_apartment_df(apartment_df)
# cleaned

In [19]:
scc.browser.get('https://www.trulia.com/for_rent/Woburn,MA/2_p/')



In [29]:
soup = BeautifulSoup(scc.browser.page_source, 'html.parser')
next_page = soup.find('a', {'aria-label': 'Next Page'})['href']

In [34]:
soup.find('a', {'aria-label': 'Next Page'})['href']

'/for_rent/Woburn,MA/3_p/'

In [40]:
scc.browser.current_url


'https://www.trulia.com/for_rent/Woburn,MA/2_p/'

In [137]:
# cleaned.astype({'sqft': 'int32', 'price': 'int32', 'bath': 'float32', 'bed': 'float32',
#                 'zipcode': 'int32'}).dtypes
# cleaned[cleaned['price'].apply(lambda x: not x.isnumeric())]
converted = scc.convert_df_columns(cleaned)

In [138]:
scc.save_to_csv(converted)

In [124]:
cleaned.date

1      2020-12-24
2      2020-12-24
4      2020-12-24
5      2020-12-24
6      2020-12-24
          ...    
686    2020-12-24
687    2020-12-24
688    2020-12-24
689    2020-12-24
690    2020-12-24
Name: date, Length: 583, dtype: object

In [5]:
 
@function_timer
def main():    

       
    

In [6]:
main()

Started Scraping
Getting URL list
0it [00:00, ?it/s]URLs retrieved: 0
Getting apartment data from url_list
Elapsed time: 0.0 minutes for <function get_all_apartments at 0x11d17faf0>
Apartments retrieved: 0
Creating DataFrame from apartment list
Elapsed time: 0.0 minutes for <function create_df at 0x11d17fc10>
Formatting Dataframe



TypeError: '(Series([], Name: sqft, dtype: bool), 'sqft')' is an invalid key